<a href="https://colab.research.google.com/github/yfpang7/dataScience_projects/blob/main/FFN_predictingMissingValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objective: Using FFN for missing data interpolation

In [333]:
import polars as pl
import polars.selectors as cs
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import copy
import sklearn.metrics as skm
import shap

pl.Config.set_tbl_cols(-1)


polars.config.Config

In [334]:
# import the wine dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

raw_data = pl.read_csv(url, separator=';', infer_schema_length=10000)

print(raw_data.shape)
print(raw_data.head())

(1599, 12)
shape: (5, 12)
┌────────┬────────┬────────┬────────┬───────┬───────┬───────┬───────┬──────┬───────┬───────┬───────┐
│ fixed  ┆ volati ┆ citric ┆ residu ┆ chlor ┆ free  ┆ total ┆ densi ┆ pH   ┆ sulph ┆ alcoh ┆ quali │
│ acidit ┆ le aci ┆ acid   ┆ al     ┆ ides  ┆ sulfu ┆ sulfu ┆ ty    ┆ ---  ┆ ates  ┆ ol    ┆ ty    │
│ y      ┆ dity   ┆ ---    ┆ sugar  ┆ ---   ┆ r dio ┆ r dio ┆ ---   ┆ f64  ┆ ---   ┆ ---   ┆ ---   │
│ ---    ┆ ---    ┆ f64    ┆ ---    ┆ f64   ┆ xide  ┆ xide  ┆ f64   ┆      ┆ f64   ┆ f64   ┆ i64   │
│ f64    ┆ f64    ┆        ┆ f64    ┆       ┆ ---   ┆ ---   ┆       ┆      ┆       ┆       ┆       │
│        ┆        ┆        ┆        ┆       ┆ f64   ┆ f64   ┆       ┆      ┆       ┆       ┆       │
╞════════╪════════╪════════╪════════╪═══════╪═══════╪═══════╪═══════╪══════╪═══════╪═══════╪═══════╡
│ 7.4    ┆ 0.7    ┆ 0.0    ┆ 1.9    ┆ 0.076 ┆ 11.0  ┆ 34.0  ┆ 0.997 ┆ 3.51 ┆ 0.56  ┆ 9.4   ┆ 5     │
│        ┆        ┆        ┆        ┆       ┆       ┆       ┆ 8  

In [335]:
# find any null values
raw_data.filter(pl.any_horizontal(pl.all().is_null() | pl.all().is_nan()))

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64


In [336]:
# add index
raw_data = raw_data.with_row_index()

# select randomly 10 rows to be set a null for 'residual sugar' the exercise
random10Rows = (
    raw_data
    .sample(10,)

)

# remove these 10 rows fromt he original dataset
data = (
    raw_data
    .join(
        random10Rows,
        on='index',
        how='anti'
    )
    .drop('index')
)
print(data.shape)
data.head()


(1589, 12)


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,6
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [337]:
# make the residual sugar column all Nulls for the extracted 10 rows - we will use this as the test for our predictions
nullSugarResidualRows = (
    random10Rows
    .with_columns(
        pl.when(pl.col('residual sugar') > 0)
        .then(None).alias('residual sugar')
    )
    .drop('index')
)
nullSugarResidualRows.head(2)

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
f64,f64,f64,null,f64,f64,f64,f64,f64,f64,f64,i64
6.6,0.63,0.0,null,0.093,51.0,77.5,0.99558,3.2,0.45,9.5,5
8.8,0.24,0.54,null,0.083,25.0,57.0,0.9983,3.39,0.54,9.2,5


In [338]:
# train using the residual data as the label data
features = data.drop('residual sugar')
label = data['residual sugar']

In [339]:
features.schema

Schema([('fixed acidity', Float64),
        ('volatile acidity', Float64),
        ('citric acid', Float64),
        ('chlorides', Float64),
        ('free sulfur dioxide', Float64),
        ('total sulfur dioxide', Float64),
        ('density', Float64),
        ('pH', Float64),
        ('sulphates', Float64),
        ('alcohol', Float64),
        ('quality', Int64)])

In [340]:
# scale and split
scaler = StandardScaler()

scaled_features = scaler.fit_transform(features)

scaled_features_df = pl.DataFrame(scaled_features, schema=features.columns)

# check the distribution of the data
fig = px.box(
    data_frame=scaled_features_df.unpivot(variable_name='features'),
    x='features',
    y='value',
    points='all'
)

fig.update_layout(template='ggplot2', width=800)
fig.show()

In [341]:
fig = px.imshow(
    scaled_features_df.with_columns(data['residual sugar']).corr(),
    y=scaled_features_df.with_columns(data['residual sugar']).columns,
    text_auto='.1f',
    zmin=-1,
    zmax=1,
    color_continuous_scale='RdBu'
)

fig.update_xaxes(side='top')
fig.update_layout(template='ggplot2', width=700, height=700)

## Data Preparation

In [342]:
# split the data and normalize
X_train, X_test, y_train, y_test = train_test_split(
    features.to_numpy(),
    label.to_numpy(),
    test_size=0.3,
    random_state=42,
    )

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [343]:
# convert to tensors and create data loaders
# cross entropy expects long or torch.int64
# regression needs torch.float32
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [344]:
# convert to pytorch dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [345]:
# translate to dataloader
batchSize = 16
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=test_dataset.tensors[0].shape[0])

## Create Model and Components

In [346]:
class ANNModel(nn.Module):
    def __init__(self, input_size, hidden_size, n_hidden_layers,
                 output_size=1, activation="relu", output_activation=None,
                 dropout=0.0, input_dropout=False):
        super().__init__()

        activations = {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "sigmoid": nn.Sigmoid(),
            "leakyrelu": nn.LeakyReLU(),
        }
        # default nn.ReLU() if none found
        self.activation = activations.get(activation.lower(), nn.ReLU())

        layers = []

        # Input layer
        layers.append(nn.Linear(input_size, hidden_size))
        if input_dropout and dropout > 0:
            layers.append(nn.Dropout(dropout))

        # Hidden layers
        for _ in range(n_hidden_layers):
            layers.append(nn.Linear(hidden_size, hidden_size))
            if dropout > 0:
                layers.append(nn.Dropout(dropout))

        # Output layer
        layers.append(nn.Linear(hidden_size, output_size))

        self.layers = nn.ModuleList(layers)
        self.output_activation = output_activation

    def forward(self, x):
        # Process all layers except output
        # check for linear layer (a fully connected)
        # if this layer object an instance of the nn.Linear()
        for layer in self.layers[:-1]:
            if isinstance(layer, nn.Linear):
                x = self.activation(layer(x))
            else:  # Dropout
                # if anything else just pass the data through the dropout layer
                x = layer(x)

        # Output layer (no activation on linear part)
        x = self.layers[-1](x)

        # Apply output activation if specified
        if self.output_activation:
            if self.output_activation.lower() == "sigmoid":
                x = torch.sigmoid(x)
            elif self.output_activation.lower() == "softmax":
                x = F.softmax(x, dim=-1)
            # Could add more activations here

        return x

In [347]:
# test the class
ANNModel(
    input_size=20,
    hidden_size=64,
    n_hidden_layers=2,
    output_size=1,              # Binary classification
    activation="relu",          # Hidden layer activation
    output_activation="sigmoid",# Output activation
    dropout=0.3,                # 30% dropout
    input_dropout=True          # Apply dropout on input too
)


ANNModel(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): Dropout(p=0.3, inplace=False)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [348]:
def create_ann_network(
    input_size,
    hidden_size,  # renamed from nUnits for clarity
    n_hidden_layers,  # renamed from nLayers for clarity
    learning_rate=0.01,
    task="regression",   # "regression", "binary", "multiclass"
    num_classes=2,       # default to 2 for binary
    activation="relu",
    dropout=0.0,
    optimizer_type="adam",
    scheduler_step=10,
    scheduler_gamma=0.95
):
    """
    Create a complete ANN setup with model, loss, optimizer, and scheduler.

    Args:
        input_size: Number of input features
        hidden_size: Number of units in hidden layers
        n_hidden_layers: Number of hidden layers (excluding input/output)
        learning_rate: Learning rate for optimizer
        task: Type of task - "regression", "binary", or "multiclass"
        num_classes: Number of classes (for multiclass only)
        activation: Activation function for hidden layers
        dropout: Dropout probability
        optimizer_type: Type of optimizer ("adam", "sgd", "rmsprop")
        scheduler_step: Step size for learning rate scheduler
        scheduler_gamma: Gamma for learning rate scheduler

    """

    # Configure output settings based on task
    if task == "regression":
        output_size = 1
        output_activation = None
        loss_func = nn.MSELoss()

    elif task == "binary":
        output_size = 1
        output_activation = None  # Use logits with BCEWithLogitsLoss
        loss_func = nn.BCEWithLogitsLoss()  # More numerically stable

    elif task == "multiclass":
        output_size = num_classes
        output_activation = None  # Use logits with CrossEntropyLoss
        loss_func = nn.CrossEntropyLoss()

    else:
        raise ValueError("task must be 'regression', 'binary', or 'multiclass'")

    # Build model
    model = ANNModel(
        input_size=input_size,
        hidden_size=hidden_size,
        n_hidden_layers=n_hidden_layers,
        output_size=output_size,
        activation=activation,
        output_activation=output_activation,
        dropout=dropout
    )

    # Create optimizer
    optimizers = {
        "adam": torch.optim.Adam,
        "sgd": torch.optim.SGD,
        "rmsprop": torch.optim.RMSprop
    }

    optimizer_class = optimizers.get(optimizer_type.lower(), torch.optim.Adam)
    optimizer = optimizer_class(model.parameters(), lr=learning_rate)

    # Create scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=scheduler_step,
        gamma=scheduler_gamma
    )

    return model, loss_func, optimizer, scheduler

In [349]:
# define a function for trianing
def trainModel(
    train_loader,
    test_loader,
    input_size,
    hidden_size=32,
    n_hidden_layers=2,
    learning_rate=0.01,
    task="regression",
    num_classes=2,
    activation="relu",
    dropout=0.0,
    optimizer_type="adam",
    scheduler_step=10,
    scheduler_gamma=0.95,
    numepochs=100,
    toggleDynamic=False
):

  training_progress = pl.DataFrame(
      schema=[
          ('iteration', pl.Int64),
          ('epoch', pl.Int64),
          ('train_loss', pl.Float64),
          ('train_acc', pl.Float64),
          ('test_loss', pl.Float64),
          ('test_acc', pl.Float64),
          ('learning_rate', pl.Float64)
      ]
  )
  iteration = 0
  # create the ann model
  model, loss_func, optimizer, scheduler = create_ann_network(
    input_size,
    hidden_size,
    n_hidden_layers,
    learning_rate,
    task,
    num_classes,
    activation,
    dropout,
    optimizer_type,
    scheduler_step,
    scheduler_gamma,
  )

  # early stopping parameters
  patience =  15
  best_loss = float('inf')
  no_improve_count = 0
  best_model = None

  # training loop
  for epoch in range(numepochs):
    model.train()
    epoch_train_loss = 0
    epoch_train_acc = 0
    num_batches = 0

    # iterate through the training batches
    for X_trainTL, y_trainTL in train_loader:
      yhat_train = model(X_trainTL)


      # Handle different task types for loss calculation
      if task == "multiclass":
        train_loss = loss_func(yhat_train, y_trainTL)
      else:
        train_loss = loss_func(yhat_train.squeeze(), y_trainTL.float())

      ## backpropagation
      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()
      ## evaluate training accuracy
      if task == "multiclass":
      # For multiclass, use predicted class
        predicted = torch.argmax(yhat_train, dim=1)
        train_acc = torch.mean((predicted == y_trainTL).float()) * 100
      elif task == "binary":
      # For binary classification with logits
        predicted = (torch.sigmoid(yhat_train.squeeze()) > 0.5).float()
        train_acc = torch.mean((predicted == y_trainTL.float()).float()) * 100
      else:
      # For regression - accuracy within tolerance
        train_acc = torch.mean((torch.abs(yhat_train.squeeze() - y_trainTL.float()) < 1).float()) * 100

      ## sum of the loss and accuracy
      epoch_train_loss += train_loss.item()
      epoch_train_acc += train_acc.item()
      num_batches += 1

      # Iteration for each batch
      iteration += 1

    ## update the learning rate after all batches (once per epoch) -  lr decay - if enabled
    if toggleDynamic:
      scheduler.step()
      current_lrs = scheduler.get_last_lr()[0]
    else:
      current_lrs = optimizer.param_groups[0]['lr']

    # evaluate test set
    model.eval()
    test_loss_total = 0
    test_acc_total = 0
    test_batches = 0

    with torch.no_grad():
      for X_testTL, y_testTL in test_loader:
        yhat_test = model(X_testTL)
        # Handle different task types for loss calculation
        if task == "multiclass":
          test_loss = loss_func(yhat_test, y_testTL)
        else:
          test_loss = loss_func(yhat_test.squeeze(), y_testTL.float())

        # Calculate test accuracy using the same metric as training
        if task == "multiclass":
        # For multiclass, use predicted class
          predicted = torch.argmax(yhat_test, dim=1)
          test_acc = torch.mean((predicted == y_testTL).float()) * 100
        elif task == "binary":
        # For binary classification with logits
          predicted = (torch.sigmoid(yhat_test.squeeze()) > 0.5).float()
          test_acc = torch.mean((predicted == y_testTL.float()).float()) * 100
        else:
        # For regression - accuracy within tolerance
          test_acc = torch.mean((torch.abs(yhat_test.squeeze() - y_testTL.float()) < 1).float()) * 100

        test_loss_total += test_loss.item()
        test_acc_total += test_acc.item()
        test_batches += 1

    # Calculate average losses and accuracies for the epoch
    avg_train_loss = epoch_train_loss / num_batches
    avg_train_acc = epoch_train_acc / num_batches
    avg_test_loss = test_loss_total / test_batches
    avg_test_acc = test_acc_total / test_batches

    # Add to training progress
    new_row = pl.DataFrame([{
        'iteration': iteration,
        'epoch': epoch,
        'train_loss': avg_train_loss,
        'train_acc': avg_train_acc,
        'test_loss': avg_test_loss,
        'test_acc': avg_test_acc,
        'learning_rate': current_lrs
    }])
    training_progress = pl.concat([training_progress, new_row])

    # Early stopping check
    if avg_test_loss < best_loss:
      best_loss = avg_test_loss
      no_improve_count = 0
      best_model = copy.deepcopy(model.state_dict())
    else:
      no_improve_count += 1

    if no_improve_count >= patience:
      print(f"Early stopping at epoch {epoch}")
      break

    # Print progress occasionally
    if epoch % 50 == 0 or epoch == numepochs - 1:
      print(f"Epoch {epoch}: Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

  # Load best model
  if best_model is not None:
    model.load_state_dict(best_model)

  return training_progress, model


In [350]:
# test the model
training_progress, trained_model = trainModel(
    train_loader=train_loader,
    test_loader=test_loader,
    input_size=11,
    hidden_size=32,
    n_hidden_layers=2,
    learning_rate=0.01,
    task="regression",
    num_classes=2, # for multiclass only
    activation="relu",
    dropout=0.0,
    optimizer_type="adam",
    scheduler_step=10,
    scheduler_gamma=0.95,
    numepochs=100,
    toggleDynamic=False
    )

Epoch 0: Train Loss: 2.4581, Test Loss: 1.8118
Early stopping at epoch 37


In [351]:
# create the prediction experiment
all_training_progress = []

for i in range(10):
  print(f'Training experiment: {i}')
  training_progress, trained_model = trainModel(
  train_loader=train_loader,
  test_loader=test_loader,
  input_size=11,
  hidden_size=32,
  n_hidden_layers=2,
  learning_rate=0.01,
  task="regression",
  num_classes=2, # for multiclass only
  activation="relu",
  dropout=0.0,
  optimizer_type="adam",
  scheduler_step=10,
  scheduler_gamma=0.95,
  numepochs=500,
  toggleDynamic=False
  )

  training_progress = training_progress.with_columns(pl.lit(i).alias('experiment_num'))

  all_training_progress.append(training_progress)

final_training_progress = pl.concat(all_training_progress)

final_training_progress.head()

Training experiment: 0
Epoch 0: Train Loss: 2.4665, Test Loss: 1.8470
Epoch 50: Train Loss: 0.6746, Test Loss: 0.9994
Early stopping at epoch 84
Training experiment: 1
Epoch 0: Train Loss: 2.7381, Test Loss: 1.9015
Epoch 50: Train Loss: 0.8227, Test Loss: 1.3761
Early stopping at epoch 68
Training experiment: 2
Epoch 0: Train Loss: 2.7281, Test Loss: 2.0498
Early stopping at epoch 42
Training experiment: 3
Epoch 0: Train Loss: 2.6873, Test Loss: 1.8058
Epoch 50: Train Loss: 0.7797, Test Loss: 0.6907
Early stopping at epoch 58
Training experiment: 4
Epoch 0: Train Loss: 2.6004, Test Loss: 2.0082
Early stopping at epoch 45
Training experiment: 5
Epoch 0: Train Loss: 2.5840, Test Loss: 1.8455
Early stopping at epoch 35
Training experiment: 6
Epoch 0: Train Loss: 2.4583, Test Loss: 1.8233
Early stopping at epoch 50
Training experiment: 7
Epoch 0: Train Loss: 2.3894, Test Loss: 1.8971
Epoch 50: Train Loss: 0.6159, Test Loss: 0.6410
Early stopping at epoch 93
Training experiment: 8
Epoch 0: 

iteration,epoch,train_loss,train_acc,test_loss,test_acc,learning_rate,experiment_num
i64,i64,f64,f64,f64,f64,f64,i32
69,0,2.466461,72.735507,1.847008,82.599579,0.01,0
138,1,1.805879,81.431159,1.733565,88.888893,0.01,0
207,2,1.755398,83.423913,1.853674,63.522011,0.01,0
276,3,1.710732,80.163043,1.69295,88.469604,0.01,0
345,4,1.586752,82.246377,1.913286,85.534592,0.01,0


In [352]:
fig = px.scatter(
    data_frame=final_training_progress,
    x='epoch',
    y=['train_loss' ,'test_loss'],
    facet_col='experiment_num',
    facet_col_wrap=3
)

fig.update_layout(template='ggplot2', width=800, height=900)
fig.update_traces(marker=dict(opacity=0.6, line=dict(width=.2, color='darkslategray')))

fig.show()

In [353]:
fig = px.scatter(
    data_frame=final_training_progress,
    x='epoch',
    y=['train_acc' ,'test_acc'],
    facet_col='experiment_num',
    facet_col_wrap=3
)

fig.update_layout(template='ggplot2', width=800, height=900)
fig.update_traces(marker=dict(opacity=0.6, line=dict(width=.2, color='darkslategray')))

fig.show()

In [354]:
# Evaluate the whole dataset
# Transform the whole feature dataset using the same scaler that was used for training
scaled_full_features = scaler.transform(features.to_numpy())

with torch.no_grad():
    trained_model.eval()
    predictions = trained_model(torch.tensor(scaled_full_features, dtype=torch.float32))

In [355]:
predictions

tensor([[2.0451],
        [2.1858],
        [2.1599],
        ...,
        [2.3109],
        [1.8077],
        [3.9116]])

In [356]:
concordance = pl.DataFrame({
    'actual_residual_sugar': label.to_numpy(),
    'predicted_residual_sugar': np.round(predictions.squeeze().numpy(), 2),
})

In [357]:
px.scatter(
    data_frame=concordance,
    x='actual_residual_sugar',
    y='predicted_residual_sugar',
    trendline='ols'
).update_layout(template='ggplot2', width=500, height=400)

In [358]:
# Calculate performance metrics on the full dataset
actual_tensor = torch.tensor(label.to_numpy(), dtype=torch.float32)
predicted_tensor = predictions.squeeze()

mse_full = torch.mean((predicted_tensor - actual_tensor) ** 2)
mae_full = torch.mean(torch.abs(predicted_tensor - actual_tensor))
r2_full = 1 - (torch.sum((actual_tensor - predicted_tensor) ** 2) /
               torch.sum((actual_tensor - torch.mean(actual_tensor)) ** 2))

print(f"MSE: {mse_full:.4f}")
print(f"MAE: {mae_full:.4f}")
print(f"R²: {r2_full:.4f}")

MSE: 0.5915
MAE: 0.4614
R²: 0.7037


In [359]:
# predict the missing data values

scaled_nulls = scaler.transform(nullSugarResidualRows.drop('residual sugar').to_numpy())

with torch.no_grad():
  trained_model.eval()
  predictions = trained_model(torch.tensor(scaled_nulls, dtype=torch.float32))


# compare the labels and predictions
pl.DataFrame({
    'actual_residual_sugar': random10Rows['residual sugar'], # the answer for the nulls (before replaced with nulls)
    'predicted_residual_sugar': np.round(predictions.squeeze().numpy(), 2),
})

actual_residual_sugar,predicted_residual_sugar
f64,f32
4.3,2.57
2.5,2.23
1.8,1.94
2.3,1.89
2.2,1.82
2.5,2.45
2.4,2.11
1.7,2.06
2.0,2.0
